# Exercise - week 2: German translator

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

The assistant will transform your spoken English to text, then translate it German and speak it out.

In [ ]:
# Install first PortAudio, in MacOS
# brew install portaudio

!pip install openai speechrecognition pyaudio


In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [8]:
system_message = """You are a highly skilled language translator specializing in translating English text to German. 
Your task is to accurately translate any English text provided by the user into German. 
Ensure that the translations are grammatically correct and contextually appropriate. 
If the user provides a phrase, sentence, or paragraph in English, respond with the equivalent translation in German.""" 

In [26]:
import speech_recognition as sr


def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something...")
        audio = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio)
        print(f"You said: {text}")
        return text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

def recognize_speech(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Google Speech Recognition could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"


def get_chatgpt_response(message):
    response = openai.chat.completions.create(
        model=MODEL,
        messages = 
            [{"role": "system", "content": system_message},
            {"role": "user", "content": message}],
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

# If problem to find microphone, upload voice file
# To record a wav-file you can use Audacity:
# brew install --cask audacity

def process_audio(audio_file):
    text = recognize_speech(audio_file)
    if text:
        response = get_chatgpt_response(text)
        return response
    return "Could not recognize speech."

# This is the microphone version:
# 
# def process_audio():
#     text = recognize_speech()
#     if text:
#         response = get_chatgpt_response(text)
#         return response
#     return "Could not recognize speech."


In [ ]:
# Create Gradio interface
iface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath"),
    outputs="text",
    live=True, 
)

if __name__ == "__main__":
    iface.launch()